# SparkSQL

Execute arbitrary SQL queries againts CSV and PARQUET files

In [ ]:
%%bash
export version=`python --version |awk '{print $2}' |awk -F"." '{print $1$2}'`

if [ $version == '36' ]; then
    pip install pyspark==2.4.8 wget==3.2 pyspark2pmml==0.5.1
elif [ $version == '38' ]; then
    pip install pyspark==3.1.2 wget==3.2 pyspark2pmml==0.5.1
else
    echo 'Currently only python 3.6 and 3.8 is supported, in case you need a different version please open an issue at https://github.com/elyra-ai/component-library/issues'
    exit -1
fi

In [ ]:
import wget
wget.download(
    'https://raw.githubusercontent.com/IBM/claimed/master/component-library/claimed_utils.py'
)
from claimed_utils import parse_args_to_parameters
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import os
import sys
import re

In [ ]:
# name of resulting file (default: data_result.csv)
output_result_file = os.environ.get('output_result_file', 'data_result.csv')

# file name for CSV or PARQUET file - must end with .csv or .parquet (default: data.csv)
data_file = os.environ.get('data_file', 'data.csv')

# master url of spark master (default: local mode)
master = os.environ.get('master', "local[*]")

# data_dir temporal data storage for local execution (default: ../../data/)
data_dir = os.environ.get('data_dir', '../../data/')

parse_args_to_parameters()

In [ ]:
sc = SparkContext.getOrCreate(SparkConf().setMaster(master))
spark = SparkSession.builder.getOrCreate()

In [ ]:
if '.csv' in data_file:
    df = spark.read.option('header', 'true').csv(data_dir + data_file)
else if '.parquet' in data_file:
    df = spark.read.parquet.csv(data_dir + data_file)
else:
    raise Exception('Neither .csv nor .parquet in file name')

In [ ]:
if os.path.exists(data_dir + output_result_file):
    shutil.rmtree(data_dir + output_result_file)

if '.csv' in data_file:
    df.coalesce(1).write.option("header", "true").csv(data_dir + output_result_file)
else if '.parquet' in data_file:
    df.coalesce(1).write.parquet(data_dir + output_result_file)
else:
    raise Exception('Neither .csv nor .parquet in file name')

file = glob.glob(data_dir + output_result_file + '/part-*')
shutil.move(file[0], data_dir + output_result_file + '.tmp')
shutil.rmtree(data_dir + output_result_file)
shutil.move(data_dir + output_result_file + '.tmp', data_dir + output_result_file)


